# Assembly: conditionals
_COSC 208, Introduction to Computer Systems, 2023-03-27_

## Announcements
* Project 3 due Thursday @ 11pm

## Outline
* Warm-up
* Mapping C conditionals to assembly code
* Transforming C code to goto-form
* Practice with conditionals

## Warm-up (Q3 & Q4 from Friday)

Q1: _The following C code was compiled into assembly (using `gcc`)._

In [ ]:
/*1*/  int divide(int numerator, int denominator) {
/*2*/      int result = -1;
/*3*/      result = numerator / denominator;
/*4*/      return result;
/*5*/  }

_For each line of assembly:_
1. _Indicate which original line of C code (above) the assembly instruction was derived from; and_
2. _Write the low-level C code equivalent of the assembly instruction, treating registers as if they were variable names._

```
000000000000076c <divide>:                      // Line     Low-level C
    76c:    d10083ff     sub    sp, sp, #0x20   // 1        sp = sp - 0x20
    770:    b9000fe0     str    w0, [sp, #12]   // 1        *(sp+12) = w0
    774:    b9000be1     str    w1, [sp, #8]    // 1        *(sp+8) = w1
    778:    12800000     mov    w0, #0xffffffff // 2        w0 = 0xffffffff
    77c:    b9001fe0     str    w0, [sp, #28]   // 2        *(sp+28) = w0
    780:    b9400fe1     ldr    w1, [sp, #12]   // 3        w1 = *(sp+12)
    784:    b9400be0     ldr    w0, [sp, #8]    // 3        w0 = *(sp+8)
    788:    1ac00c20     sdiv   w0, w1, w0      // 3        w0 = w1/w0
    78c:    b9001fe0     str    w0, [sp, #28]   // 3        *(sp+28) = w0
    790:    b9401fe0     ldr    w0, [sp, #28]   // 4        w0 = *(sp+28)
    794:    910083ff     add    sp, sp, #0x20   // 4        sp = sp + 0x20
    798:    d65f03c0     ret                    // 4        return
```

```
000000000000076c <divide>:                      // Line     Low-level C
    76c:    d10083ff     sub    sp, sp, #0x20   // 
    770:    b9000fe0     str    w0, [sp, #12]   // 
    774:    b9000be1     str    w1, [sp, #8]    // 
    778:    12800000     mov    w0, #0xffffffff // 
    77c:    b9001fe0     str    w0, [sp, #28]   // 
    780:    b9400fe1     ldr    w1, [sp, #12]   // 
    784:    b9400be0     ldr    w0, [sp, #8]    // 
    788:    1ac00c20     sdiv   w0, w1, w0      // 
    78c:    b9001fe0     str    w0, [sp, #28]   //
    790:    b9401fe0     ldr    w0, [sp, #28]   // 
    794:    910083ff     add    sp, sp, #0x20   // 
    798:    d65f03c0     ret                    // 
```

Q2: _Assume the registers initially hold the following values:_
```
sp = 0xFE0
w/x0 = 100
w/x1 = 5
```
_Draw the contents of the stack and registers just prior to the execution of the second-to-last instruction above._

```
        0  1  2  3  4  5  6  7  8          
 sp ->  +--+--+--+--+--+--+--+--+
 0xFC0  |                       |
        +--+--+--+--+--+--+--+--+
 0xFC8  |     5     |    100    |
        +--+--+--+--+--+--+--+--+
 0xFD0  |                       |
        +--+--+--+--+--+--+--+--+
 0xFD8  |           |     20    |
        +--+--+--+--+--+--+--+--+
sp = 0xFC0
w/x0 = 20
w/x1 = 100
```

<p style="height:10em;"></p>

🛑 **STOP here** after completing the above question; if you have extra time please **skip ahead** to the extra practice.

* _Why is `#0xffffffff` being stored in `w0`?_ — this is the two's complement representation of -1
* _When might this function cause an error?_ — when denominator is 0
* _How would you modify the C code to avoid an error?_

In [ ]:
/*1*/  int divide_safe(int numerator, int denominator) {
/*2*/      int result = -1;
/*3*/      if (denominator != 0) {
/*4*/          result = numerator / denominator;
/*5*/      }
/*6*/      return result;
/*7*/  }

## Mapping C conditionals to assembly code

In [ ]:
/*1*/  int divide_safe(int numerator, int denominator) {
/*2*/      int result = -1;
/*3*/      if (denominator != 0) {
/*4*/          result = numerator / denominator;
/*5*/      }
/*6*/      return result;
/*7*/  }

```
000000000000076c <divide_safe>:                     // Line
    76c:    d10083ff     sub    sp, sp, #0x20       // 1
    770:    b9000fe0     str    w0, [sp, #12]       // 1
    774:    b9000be1     str    w1, [sp, #8]        // 1
    778:    12800000     mov    w0, #0xffffffff     // 2
    77c:    b9001fe0     str    w0, [sp, #28]       // 2
    780:    b9400be0     ldr    w0, [sp, #8]        // 3
    784:    7100001f     cmp    w0, #0x0            // 3
    788:    540000a0     b.eq   79c <divide_safe+0x30> // 3
    78c:    b9400fe1     ldr    w1, [sp, #12]       // 4
    790:    b9400be0     ldr    w0, [sp, #8]        // 4
    794:    1ac00c20     sdiv   w0, w1, w0          // 4
    798:    b9001fe0     str    w0, [sp, #28]       // 4
    79c:    b9401fe0     ldr    w0, [sp, #28]       // 6
    7a0:    910083ff     add    sp, sp, #0x20       // 6
    7a4:    d65f03c0     ret                        // 6
```

```
000000000000076c <divide_safe>:
    76c:    d10083ff     sub    sp, sp, #0x20
    770:    b9000fe0     str    w0, [sp, #12]
    774:    b9000be1     str    w1, [sp, #8]
    778:    12800000     mov    w0, #0xffffffff
    77c:    b9001fe0     str    w0, [sp, #28]
    780:    b9400be0     ldr    w0, [sp, #8]
    784:    7100001f     cmp    w0, #0x0
    788:    540000a0     b.eq   79c <divide_safe+0x30>
    78c:    b9400fe1     ldr    w1, [sp, #12]
    790:    b9400be0     ldr    w0, [sp, #8]
    794:    1ac00c20     sdiv   w0, w1, w0
    798:    b9001fe0     str    w0, [sp, #28]
    79c:    b9401fe0     ldr    w0, [sp, #28]
    7a0:    910083ff     add    sp, sp, #0x20
    7a4:    d65f03c0     ret
```

* _What does the `cmp` instruction do?_ — compares a register's value to another value
* _What does the `b.eq` instruction do?_ — "jumps" (i.e., branches) to a different instruction when the compared values are equal
* _Why does the assembly check if `w0 == 0` when the C code contains `!= 0`?_ — the C code checks for the condition that must be true to execute the if body, whereas the assembly code checks for the condition that must be true to **skip over** the if body

## Transforming C code to goto-form

* _How would we express this in C code?_ — using an if statement and a `goto` statement

In [ ]:
/*1*/  int divide_safe_goto(int numerator, int denominator) {
/*2*/      int result = -1;
/*3*/      if (denominator == 0)
/*4*/          goto after;
/*5*/      result = numerator / denominator;
/*6*/  after:
/*7*/      return result;
/*8*/  }

* _What happens if the code includes an else statement?_ — if condition is true, execute the if body and skip over the else body; if condition is false, skip over the if body and execute the else body

<div style="page-break-after:always;"></div>

## Practice with conditionals

Q2: _The C code below was compiled into assembly (using `gcc`). Label each line of assembly code with the line number of the line of C code from which the assembly instruction was derived._

In [ ]:
/* 1*/  int flip(int bit) {
/* 2*/     int result = -1;
/* 3*/     if (bit == 0) {
/* 4*/          result = 1; 
/* 5*/      } 
/* 6*/      else {
/* 7*/          result = 0;
/* 8*/      }
/* 9*/      return result;
/*10*/ }

```
000000000000071c <flip>:                            // Line
    71c:    d10083ff     sub    sp, sp, #0x20       // 1
    720:    b9000fe0     str    w0, [sp, #12]       // 1
    724:    12800000     mov    w0, #0xffffffff     // 2
    728:    b9001fe0     str    w0, [sp, #28]       // 2
    72c:    b9400fe0     ldr    w0, [sp, #12]       // 3
    730:    7100001f     cmp    w0, #0x0            // 3
    734:    54000081     b.ne   744 <flip+0x28>     // 3
    738:    52800020     mov    w0, #0x1            // 4
    73c:    b9001fe0     str    w0, [sp, #28]       // 4
    740:    14000002     b      748 <flip+0x2c>     // 5
    744:    b9001fff     str    wzr, [sp, #28]      // 7
    748:    b9401fe0     ldr    w0, [sp, #28]       // 9
    74c:    910083ff     add    sp, sp, #0x20       // 9
    750:    d65f03c0     ret                        // 9
```

```
000000000000071c <flip>:                            // Line
    71c:    d10083ff     sub    sp, sp, #0x20       // 
    720:    b9000fe0     str    w0, [sp, #12]       // 
    724:    12800000     mov    w0, #0xffffffff     // 
    728:    b9001fe0     str    w0, [sp, #28]       // 
    72c:    b9400fe0     ldr    w0, [sp, #12]       // 
    730:    7100001f     cmp    w0, #0x0            //
    734:    54000081     b.ne   744 <flip+0x28>     // 
    738:    52800020     mov    w0, #0x1            // 
    73c:    b9001fe0     str    w0, [sp, #28]       // 
    740:    14000002     b      748 <flip+0x2c>     // 
    744:    b9001fff     str    wzr, [sp, #28]      // 
    748:    b9401fe0     ldr    w0, [sp, #28]       // 
    74c:    910083ff     add    sp, sp, #0x20       // 
    750:    d65f03c0     ret                        // 
```

Q3: _Write a function called `flip_goto` that behaves the same as `flip` but matches the structure of the assembly code that will be generated for `flip`. (Hint: you'll need two `goto` statements.)_

In [ ]:
int flip_goto(int bit) {
    int result = -1;
    if (bit != 0)
        goto else_body;
    result = 1; 
    goto after_else;
else_body:
    result = 0;
after_else:
    return result;
}

<div style="page-break-after:always;"></div>

## Various forms of conditionals

* Else if

In [ ]:
/* 1*/  int flip_elif(int bit) {
/* 2*/      int result;
/* 3*/      if (bit == 0) {
/* 4*/          result = 1 ;
/* 5*/      }
/* 6*/      else if (bit == 1) {
/* 7*/          result = 0;
/* 8*/      }
/* 9*/      else {
/*10*/          result = -1;
/*11*/      }
/*12*/      return result;
/*13*/  }

```
000000000000071c <flip_elif>:
    71c:    d10083ff     sub    sp, sp, #0x20           // 1
    720:    b9000fe0     str    w0, [sp, #12]           // 1
    724:    b9400fe0     ldr    w0, [sp, #12]           // 3
    728:    7100001f     cmp    w0, #0x0                // 3
    72c:    54000081     b.ne   73c <flip_elif+0x20>    // 3
    730:    52800020     mov    w0, #0x1                // 4
    734:    b9001fe0     str    w0, [sp, #28]           // 4
    738:    14000008     b      758 <flip_elif+0x3c>    // 5
    73c:    b9400fe0     ldr    w0, [sp, #12]           // 6
    740:    7100041f     cmp    w0, #0x1                // 6
    744:    54000061     b.ne   750 <flip_elif+0x34>    // 6
    748:    b9001fff     str    wzr, [sp, #28]          // 7
    74c:    14000003     b      758 <flip_elif+0x3c>    // 8
    750:    12800000     mov    w0, #0xffffffff         // 10
    754:    b9001fe0     str    w0, [sp, #28]           // 10
    758:    b9401fe0     ldr    w0, [sp, #28]           // 12
    75c:    910083ff     add    sp, sp, #0x20           // 12
    760:    d65f03c0     ret                            // 12
```

* Complex conditional expressions

In [ ]:
/* 1*/  int bitwise_and(int bitA, int bitB) {
/* 2*/      int result = -1;
/* 3*/      if (bitA == 1 && bitB == 1) {
/* 4*/          result = 1;
/* 5*/      } 
/* 6*/      else {
/* 7*/          result = 0;
/* 8*/      }
/* 9*/      return result;
/*10*/  }

```
0000000000000764 <bitwise_and>:
    764:    d10083ff     sub    sp, sp, #0x20           // 1
    768:    b9000fe0     str    w0, [sp, #12]           // 1
    76c:    b9000be1     str    w1, [sp, #8]            // 1
    770:    12800000     mov    w0, #0xffffffff         // 2
    774:    b9001fe0     str    w0, [sp, #28]           // 2
    778:    b9400fe0     ldr    w0, [sp, #12]           // 3
    77c:    7100041f     cmp    w0, #0x1                // 3
    780:    540000e1     b.ne   79c <bitwise_and+0x38>  // 3
    784:    b9400be0     ldr    w0, [sp, #8]            // 3
    788:    7100041f     cmp    w0, #0x1                // 3
    78c:    54000081     b.ne   79c <bitwise_and+0x38>  // 3
    790:    52800020     mov    w0, #0x1                // 4
    794:    b9001fe0     str    w0, [sp, #28]           // 4
    798:    14000002     b      7a0 <bitwise_and+0x3c>  // 5
    79c:    b9001fff     str    wzr, [sp, #28]          // 7
    7a0:    b9401fe0     ldr    w0, [sp, #28]           // 9
    7a4:    910083ff     add    sp, sp, #0x20           // 9
    7a8:    d65f03c0     ret                            // 9
```

* Nested conditionals

In [ ]:
/* 1*/  int bitwise_and_nested(int bitA, int bitB) {
/* 2*/      int result = -1;
/* 3*/      if (bitA == 1) {
/* 4*/          if (bitB == 1) {
/* 5*/              result = 1;
/* 6*/          }
/* 7*/          else {
/* 8*/              result = 0;
/* 9*/          }
/*10*/      }
/*11*/      else {
/*12*/          result = 0;
/*13*/      }
/*14*/      return result;
/*15*/  }

```
00000000000007ac <bitwise_and_nested>:
    7ac:    d10083ff     sub    sp, sp, #0x20       // 1
    7b0:    b9000fe0     str    w0, [sp, #12]       // 1
    7b4:    b9000be1     str    w1, [sp, #8]        // 1
    7b8:    12800000     mov    w0, #0xffffffff     // 2
    7bc:    b9001fe0     str    w0, [sp, #28]       // 2
    7c0:    b9400fe0     ldr    w0, [sp, #12]       // 3
    7c4:    7100041f     cmp    w0, #0x1            // 3
    7c8:    54000121     b.ne   7ec <bitwise_and_nested+0x40>   // 3
    7cc:    b9400be0     ldr    w0, [sp, #8]        // 4
    7d0:    7100041f     cmp    w0, #0x1            // 4
    7d4:    54000081     b.ne   7e4 <bitwise_and_nested+0x38>   // 4
    7d8:    52800020     mov    w0, #0x1            // 5
    7dc:    b9001fe0     str    w0, [sp, #28]       // 5
    7e0:    14000004     b      7f0 <bitwise_and_nested+0x44>   // 6
    7e4:    b9001fff     str    wzr, [sp, #28]      // 8
    7e8:    14000002     b      7f0 <bitwise_and_nested+0x44>   // 10
    7ec:    b9001fff     str    wzr, [sp, #28]      // 12
    7f0:    b9401fe0     ldr    w0, [sp, #28]       // 14
    7f4:    910083ff     add    sp, sp, #0x20       // 14
    7f8:    d65f03c0     ret                        // 14
```

* Ternary expressions

In [ ]:
/*1*/   int bitwise_and_ternary(int bitA, int bitB) {
/*2*/       return (bitA == 1 && bitB == 1 ? 1 : 0);
/*3*/   }

```
00000000000007fc <bitwise_and_ternary>:
    7fc:    d10043ff     sub    sp, sp, #0x10   // 1
    800:    b9000fe0     str    w0, [sp, #12]   // 1
    804:    b9000be1     str    w1, [sp, #8]    // 1
    808:    b9400fe0     ldr    w0, [sp, #12]   // 2
    80c:    7100041f     cmp    w0, #0x1        // 2
    810:    540000c1     b.ne   828 <bitwise_and_ternary+0x2c>  // 2
    814:    b9400be0     ldr    w0, [sp, #8]    // 2
    818:    7100041f     cmp    w0, #0x1        // 2
    81c:    54000061     b.ne   828 <bitwise_and_ternary+0x2c>  // 2
    820:    52800020     mov    w0, #0x1        // 2
    824:    14000002     b      82c <bitwise_and_ternary+0x30>  // 2
    828:    52800000     mov    w0, #0x0        // 2
    82c:    910043ff     add    sp, sp, #0x10   // 2
    830:    d65f03c0     ret                    // 2
```

* Boolean expressions

In [ ]:
/*1*/ int bitwise_and_single(int bitA, int bitB) {
/*2*/    return (bitA == 1 && bitB == 1);
/*3*/ }

    Exact same assembly code as bitwise_and_ternary

* With compiler optimizations all four `bitwise_and` functions result in the exact same assembly code

```
0000000000000734 <bitwise_and>:
    734:    7100041f     cmp    w0, #0x1
    738:    7a410820     ccmp    w1, #0x1, #0x0, eq
    73c:    1a9f17e0     cset    w0, eq
    740:    d65f03c0     ret
```

## Extra practice

Q4: _Assume the registers currently hold the following values:_
```
sp = 0xA980
w/x0 = 0
w/x1 = 1
w/x2 = 2
w/x3 = 3
w/x4 = 4
w/x5 = 5
```
_Draw the contents of the stack after the following instructions have been executed:_
```
sub sp, sp, #0x30
str w0, [sp, #16]
str x1, [sp]
str w2, [sp, #20]
str x3, [sp, #32]
str w4, [sp, #28]
str w5, [sp, #8]
```

```
        0  1  2  3  4  5  6  7  8          
 sp ->  +--+--+--+--+--+--+--+--+
0xA950  |           1           |
        +--+--+--+--+--+--+--+--+
0xA958  |     5     |           |
        +--+--+--+--+--+--+--+--+
0xA960  |     0     |     2     |
        +--+--+--+--+--+--+--+--+
0xA968  |           |     4     |
        +--+--+--+--+--+--+--+--+
0xA970  |           3           |
        +--+--+--+--+--+--+--+--+
0xA978  |                       |
        +--+--+--+--+--+--+--+--+
0xA980  |                       |
        +--+--+--+--+--+--+--+--+
```

<p style="height:20em;"></p>

Q5: _Write a function called `adjust_goto` that behaves the same as `adjust` but matches the structure of the assembly code that will be generated for `adjust`. (Hint: you'll need two `goto` statements.)_

In [ ]:
int adjust(int value) {
    if (value < 10) {
        value = value * 10;
    }
    else {
        value = value / 10;
    }
    return value;
}

In [ ]:
int adjust_goto(int value) {
    if (value >= 10)
        goto else_body;
    value = value * 10;
    goto after_if;
else_body:
    value = value / 10;
after_if:
    return value;
}